In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
dataURL = "https://s3-us-west-1.amazonaws.com/isc.homebucket/interviews/insights/mlb_salaries.csv"
mlb_data = pd.read_csv(dataURL)
mlb_data.columns = mlb_data.columns.str.lower()

In [3]:
mlb_data.head()

,year,team,city,league,player,salary
0,1985,Braves,Atlanta,National,barkele01,870000
1,1985,Braves,Atlanta,National,bedrost01,550000
2,1985,Braves,Atlanta,National,benedbr01,545000
3,1985,Braves,Atlanta,National,campri01,633333
4,1985,Braves,Atlanta,National,ceronri01,625000


## Question 1. Average Player Salaries
Player salaries notoriously seem to increase every year, but to what extent?

a. Calculate the average player salaries by year (X-axis: Year, Y-axis: Avg. Player Salary)

b. Which year had the highest average player salary?

In [13]:
salary_by_year = mlb_data.groupby('year').agg(average_salary=('salary', np.mean))
# salary_by_year = mlb_data.groupby(['year'], as_index=False).agg(average_salary=('salary', np.mean))
# salary_by_year = mlb_data.groupby(['year']).mean()
salary_by_year.reset_index(inplace = True)
salary_by_year.head()

,year,average_salary
0,1985,476299.45
1,1986,417147.04
2,1987,434729.47
3,1988,453171.08
4,1989,506323.08


In [14]:
max_salary_index = salary_by_year['average_salary'].idxmax()
salary_by_year.loc[max_salary_index]

year                2015.00
average_salary   4301276.09
Name: 30, dtype: float64

## Question 2. Salary Trends
Is there ever a point where a player’s salary starts to decline?

a. Calculate players’ salary change based on their tenure in the league (X-axis: Years in
league, Y-axis: Percent salary change)

b. Is there a year in a player’s career where their salary starts to decline?


In [15]:
mlb_data.sort_values(by=['year'], inplace=True)
mlb_data['nth_year'] = mlb_data.groupby(['player']).cumcount() + 1
mlb_data.reset_index()

,index,year,team,city,league,player,salary,nth_year
0,0,1985,Braves,Atlanta,National,barkele01,870000,1
1,375,1985,A's,Oakland,American,hendest01,150000,1
2,374,1985,A's,Oakland,American,heathmi02,425000,1
3,373,1985,A's,Oakland,American,griffal01,610000,1
4,372,1985,A's,Oakland,American,davismi02,305000,1
5,371,1985,A's,Oakland,American,collida02,700000,1
6,370,1985,A's,Oakland,American,codirch01,141500,1
7,369,1985,A's,Oakland,American,bochtbr01,350000,1
8,368,1985,A's,Oakland,American,birtsti01,60000,1
9,367,1985,A's,Oakland,American,bakerdu01,575000,1


In [16]:
def get_pc(row):
    return (row['salary'] - row['salary'].shift()) * 100 / row['salary'].shift()

In [17]:
salary_pc = mlb_data.groupby('player').apply(get_pc)
# mlb_data['salary_pc'] = (mlb_data['salary'] - mlb_data.shift(periods=1)['salary']) * 100 / mlb_data.shift(periods=1)['salary']
# mlb_data['salary_pc'] = mlb_data.apply(lambda row: 
#                                        (row['salary'] - row['salary'].shift()) * 100 / row['salary'].shift()
#                                       )

In [18]:
mlb_data['salary_pc'] = salary_pc.reset_index(level=0, drop=True)

In [19]:
will_data = mlb_data[mlb_data['player'] == 'clarkwi02']
will_data

,year,team,city,league,player,salary,nth_year,salary_pc
1185,1986,Giants,San Francisco,National,clarkwi02,60000,1,nan
1836,1987,Giants,San Francisco,National,clarkwi02,120000,2,100.00
2481,1988,Giants,San Francisco,National,clarkwi02,320000,3,166.67
3178,1989,Giants,San Francisco,National,clarkwi02,1125000,4,251.56
4033,1990,Giants,San Francisco,National,clarkwi02,2250000,5,100.00
4746,1991,Giants,San Francisco,National,clarkwi02,3750000,6,66.67
5496,1992,Giants,San Francisco,National,clarkwi02,4275000,7,14.00
6412,1993,Giants,San Francisco,National,clarkwi02,4750000,8,11.11
7360,1994,Rangers,Arlington,American,clarkwi02,3875972,9,-18.40
8340,1995,Rangers,Arlington,American,clarkwi02,5647726,10,45.71


In [20]:
pc_by_nth_year = mlb_data.groupby('nth_year').agg(ave_salery_pc=('salary_pc', np.mean))
pc_by_nth_year = pc_by_nth_year.reset_index()
pc_by_nth_year

,nth_year,ave_salery_pc
0,1,nan
1,2,37.68
2,3,66.87
3,4,141.27
4,5,93.62
5,6,54.37
6,7,inf
7,8,23.71
8,9,21.62
9,10,21.35


In [21]:
pc_neg_only = pc_by_nth_year[pc_by_nth_year['ave_salery_pc'] < 0]
pc_neg_only[pc_neg_only['nth_year'] == pc_neg_only['nth_year'].min()]
# salary_by_year.loc[max_salary_inxdex]

,nth_year,ave_salery_pc
16,17,-0.72


In [22]:
pc_by_nth_year['nth_year'].min()

1

## Question 3. Salary Variance
There is no league-imposed team salary cap in baseball, so some teams pay considerably more
than others.

a. Calculate the standard deviation in team salary over the last 30 years (X-axis: Year,
Y-axis: Std dev of team salaries)

b. Is there a trend in the standard deviation?


In [23]:
mlb_data.head()

,year,team,city,league,player,salary,nth_year,salary_pc
0,1985,Braves,Atlanta,National,barkele01,870000,1,nan
375,1985,A's,Oakland,American,hendest01,150000,1,nan
374,1985,A's,Oakland,American,heathmi02,425000,1,nan
373,1985,A's,Oakland,American,griffal01,610000,1,nan
372,1985,A's,Oakland,American,davismi02,305000,1,nan


In [33]:
salaries_by_team = mlb_data.groupby(['year', 'team']).agg(
#     {'salary': [sum]}
    team_salary=('salary', sum)
)

team_salary_sd_by_year = salaries_by_team.groupby('year').agg(np.std)
team_salary_sd_by_year.reset_index(inplace=True)
team_salary_sd_by_year.head()


,year,team_salary
0,1985,2470845.43
1,1986,3186956.42
2,1987,3848336.54
3,1988,3386330.58
4,1989,3568844.14
